In [3]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.patches as patches
import cmcrameri.cm as cmc
import random
from tqdm.notebook import tqdm
from geopy.distance import geodesic



In [4]:
creepmeters = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv')
creepmeters.drop_duplicates('Creepmeter_abbrv',inplace=True)
creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='XMBC'].index,inplace=True)
creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='TABC'].index,inplace=True)
creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='CHP1'].index,inplace=True)
#creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='CTM1'].index,inplace=True)
#creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='XHR3'].index,inplace=True)
#creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='SH30'].index,inplace=True) #needs checking as defintely picks




creepmeters.reset_index(inplace=True,drop=True)
creepmeters_picks = pd.read_csv('../../Data/DATA_tidied/Picks/All_picks_29_May_2025.csv',index_col=0)
creepmeters_picks['Start Time'] = pd.to_datetime(creepmeters_picks['Start Time'])
creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Start Time']>dt.datetime(2024,1,1,0,0,0)].index,inplace=True)
creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Displacement, mm']<0.02].index,inplace=True)
networks = creepmeters['Network'].unique()
networks

array(['EAF', 'PARK', 'UTA', 'NAF', 'HAY', 'CHAF', 'HOL', 'DSF', 'SOCAL',
       'RID', 'CAL'], dtype=object)

In [5]:
# Compute epicentral distance and expected surface wave travel time
def compute_epicentral_distance_and_travel_time(row):
    creep_coords = (row['Latitude_creep'], row['Longitude_creep'])
    eq_coords = (row['latitude'], row['longitude'])
    distance_km = geodesic(creep_coords, eq_coords).km
    travel_time = distance_km / 3.5  # assume 3.5 km/s surface wave
    return pd.Series({'distance_km': distance_km, 'travel_time_surf_wave': travel_time})

In [ ]:
my_data = pd.DataFrame()
percentage_triggered_all = pd.DataFrame()
for j in range(len(networks)):
    print(networks[j])
    if networks[j] in ['EAF', 'PARK', 'UTA', 'NAF', 'HAY', 'CHAF', 'HOL', 'DSF', 'SOCAL','RID', 'CAL']:
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[j]].index)
        for i in range(len(creepmeters_network)):#len(creepmeters)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]
            print(abbrv)
            
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation']!=abbrv].index)
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv,lat,long)
            sampling_freq = picks['Sampling rate, s']


            # USGS API query parameters
            url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
            params = {
                "format": "geojson",
                "starttime": "1980-01-01",
                "endtime": "2024-01-01",
                "latitude": lat,
                "longitude": long,
                "maxradiuskm": 500,
                "minmagnitude": 4
            }

            response = requests.get(url, params=params)
            data = response.json()

            # Extract earthquake metadata
            eq_data = []
            for feature in data['features']:
                time = datetime.utcfromtimestamp(feature['properties']['time'] / 1000.0)
                magnitude = feature['properties']['mag']
                place = feature['properties']['place']
                lon_eq, lat_eq, depth = feature['geometry']['coordinates']  # [lon, lat, depth]
                eq_data.append({
                    'eq_time': time,
                    'magnitude': magnitude,
                    'latitude': lat_eq,
                    'longitude': lon_eq,
                    'depth_km': depth,
                    'place': place
                })

            # Create DataFrame
            eq_df = pd.DataFrame(eq_data)


            # Ensure both time columns are datetime
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            eq_df['eq_time'] = pd.to_datetime(eq_df['eq_time'])

            # Sort both by time (important!)
            picks = picks.sort_values('Start Time').reset_index(drop=True)
            eq_df = eq_df.sort_values('eq_time').reset_index(drop=True)

            # Step 1: Use merge_asof twice — once backward and once forward
            df_before = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='backward', suffixes=('', '_before'))
            # Forward merge
            df_after = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='forward')

            # Rename the columns to prevent confusion
            df_after = df_after.rename(columns={
                'eq_time': 'eq_time_after',
                'magnitude': 'magnitude_after',
                'latitude': 'latitude_after',
                'longitude': 'longitude_after',
                'depth_km': 'depth_km_after',
                'place': 'place_after'
            })


            # Step 2: Calculate time differences for each
            df_before['dt_before'] = (df_before['Start Time'] - df_before['eq_time']).dt.total_seconds().abs()
            df_after['dt_after'] = (df_after['Start Time'] - df_after['eq_time_after']).dt.total_seconds().abs()

            print("Picks with no backward match:", df_before['eq_time'].isna().sum())
            print("Picks with no forward match:", df_after['eq_time_after'].isna().sum())
            print(len(picks))

            # Step 3: Choose the closer one
            nearest_eq = []
            for i in range(len(picks)):
                dt_b = df_before.loc[i, 'dt_before']
                dt_a = df_after.loc[i, 'dt_after']
                
                # Use backward if forward is missing or farther
                if pd.isna(dt_a) or (not pd.isna(dt_b) and dt_b <= dt_a):
                    row = df_before.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time'],
                        'magnitude': row['magnitude'],
                        'latitude': row['latitude'],
                        'longitude': row['longitude'],
                        'depth_km': row['depth_km'],
                        'place': row['place'],
                        'time_diff_seconds': row['dt_before']
                    })
                else:
                    row = df_after.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time_after'],
                        'magnitude': row['magnitude_after'],
                        'latitude': row['latitude_after'],
                        'longitude': row['longitude_after'],
                        'depth_km': row['depth_km_after'],
                        'place': row['place_after'],
                        'time_diff_seconds': row['dt_after']
                    })

            # Step 4: Convert to DataFrame
            nearest_eq_df = pd.DataFrame(nearest_eq)

            # Add time difference column
            nearest_eq_df['time_since_eq_seconds'] = (nearest_eq_df['Start Time'] - nearest_eq_df['eq_time']).dt.total_seconds()

            # Threshold in seconds (1 day)
            one_day_seconds = 86400
            one_hour_seconds = 3600

            # Count how many are within 1 day
            count_within_day = (abs(nearest_eq_df['time_since_eq_seconds']) < one_day_seconds).sum()
            count_within_hour = (abs(nearest_eq_df['time_since_eq_seconds']) < one_hour_seconds).sum()

            # Total number of entries
            total_count = len(picks)

            # Percentage
            percent_within_day = 100 * count_within_day / total_count
            percent_within_hour = 100 * count_within_hour / total_count

            #print(f"{percent_within_day:.2f}% of {total_count} events occur within 1 day of an earthquake.")
            print(f"{percent_within_hour:.2f}% of {total_count} events occur within 1 hour of an earthquake.")

            percentage_triggered = pd.DataFrame({'Creepmeter':[abbrv],'percentage in day':[round(percent_within_day,2)],'percentage in hour':[round(percent_within_hour,2)],
                                                    'Latitude':lat})
            
            if abbrv in ['XHR1','XHR2','XHR3']:
                boolarr= nearest_eq_df['time_since_eq_seconds']>one_hour_seconds
                print(nearest_eq_df[boolarr])

            creepmeter_eq_picks = pd.concat([picks.reset_index(drop=True), nearest_eq_df.reset_index(drop=True)], axis=1)
            my_data=pd.concat([my_data,creepmeter_eq_picks])

            percentage_triggered_all = pd.concat([percentage_triggered_all,percentage_triggered])
        percentage_triggered_all.sort_values(by='Latitude',inplace=True,ascending=False)
        percentage_triggered_all.reset_index(inplace=True,drop=True)

EAF
BAL1
BAL1 37.9903 38.199


KeyboardInterrupt: 

In [8]:
from geopy.distance import geodesic

# Containers to store all output across networks
my_data = pd.DataFrame()
percentage_triggered_all = pd.DataFrame()

# Loop over each network in your list
for j in range(len(networks)):
    print(networks[j])

    # Limit processing to specific networks of interest
    if networks[j] in ['EAF', 'PARK', 'UTA', 'NAF', 'HAY', 'CHAF', 'HOL', 'DSF', 'SOCAL', 'RID', 'CAL']:
        
        # Filter creepmeters to just those in this network
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network'] != networks[j]].index)
        
        # Loop over each creepmeter in the network
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]
            print(abbrv)

            # Filter picks (creep events) for this creepmeter
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation'] != abbrv].index)

            # Get creepmeter location
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv, lat, long)

            # Extract per-event sampling interval (in seconds)
            sampling_freq = picks['Sampling rate, s']

            # Set up query to USGS earthquake API
            url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
            params = {
                "format": "geojson",
                "starttime": "1980-01-01",
                "endtime": "2024-01-01",
                "latitude": lat,
                "longitude": long,
                "maxradiuskm": 500,
                "minmagnitude": 4
            }

            # Query USGS and extract results
            response = requests.get(url, params=params)
            data = response.json()

            # Parse earthquake metadata into DataFrame
            eq_data = []
            for feature in data['features']:
                time = datetime.utcfromtimestamp(feature['properties']['time'] / 1000.0)
                magnitude = feature['properties']['mag']
                place = feature['properties']['place']
                lon_eq, lat_eq, depth = feature['geometry']['coordinates']
                eq_data.append({
                    'eq_time': time,
                    'magnitude': magnitude,
                    'latitude': lat_eq,
                    'longitude': lon_eq,
                    'depth_km': depth,
                    'place': place
                })

            eq_df = pd.DataFrame(eq_data)

            # Ensure times are datetime and sort for merge_asof
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            eq_df['eq_time'] = pd.to_datetime(eq_df['eq_time'])
            picks = picks.sort_values('Start Time').reset_index(drop=True)
            eq_df = eq_df.sort_values('eq_time').reset_index(drop=True)

            # Find closest earthquake before and after each pick
            df_before = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='backward', suffixes=('', '_before'))
            df_after = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='forward')

            # Rename after-EQ columns for clarity
            df_after = df_after.rename(columns={
                'eq_time': 'eq_time_after',
                'magnitude': 'magnitude_after',
                'latitude': 'latitude_after',
                'longitude': 'longitude_after',
                'depth_km': 'depth_km_after',
                'place': 'place_after'
            })

            # Calculate time differences for nearest EQs
            df_before['Time Diff to Prev EQ [s]'] = (df_before['Start Time'] - df_before['eq_time']).dt.total_seconds().abs()
            df_after['Time Diff to Next EQ [s]'] = (df_after['Start Time'] - df_after['eq_time_after']).dt.total_seconds().abs()

            print("Picks with no backward match:", df_before['eq_time'].isna().sum())
            print("Picks with no forward match:", df_after['eq_time_after'].isna().sum())
            print(len(picks))

            # Choose the closest EQ (before or after) per event
            nearest_eq = []
            for i in range(len(picks)):
                dt_b = df_before.loc[i, 'Time Diff to Prev EQ [s]']
                dt_a = df_after.loc[i, 'Time Diff to Next EQ [s]']

                # Prefer backward unless forward is closer or only one exists
                if pd.isna(dt_a) or (not pd.isna(dt_b) and dt_b <= dt_a):
                    row = df_before.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time'],
                        'magnitude': row['magnitude'],
                        'latitude': row['latitude'],
                        'longitude': row['longitude'],
                        'depth_km': row['depth_km'],
                        'place': row['place'],
                        'Time Diff to Nearest EQ [s]': row['Time Diff to Prev EQ [s]']
                    })
                else:
                    row = df_after.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time_after'],
                        'magnitude': row['magnitude_after'],
                        'latitude': row['latitude_after'],
                        'longitude': row['longitude_after'],
                        'depth_km': row['depth_km_after'],
                        'place': row['place_after'],
                        'Time Diff to Nearest EQ [s]': row['Time Diff to Next EQ [s]']
                    })

            # Create DataFrame from nearest EQ matches
            nearest_eq_df = pd.DataFrame(nearest_eq)

            # Add creepmeter location + sampling frequency for use below
            nearest_eq_df['Latitude_creep'] = lat
            nearest_eq_df['Longitude_creep'] = long
            # -- Add Sampling Interval to nearest_eq_df --
            nearest_eq_df['Sampling Interval [s]'] = picks['Sampling rate, s'].values

            # -- Compute Epicentral Distance and Surface Wave Travel Time --
            nearest_eq_df[['Epicentral Distance [km]', 'Surface Wave Travel Time [s]']] = nearest_eq_df.apply(
                compute_epicentral_distance_and_travel_time, axis=1
            )

            # -- Compute Time Since Earthquake --
            nearest_eq_df['Time Since EQ [s]'] = (nearest_eq_df['Start Time'] - nearest_eq_df['eq_time']).dt.total_seconds()

            # -- Define Noise Tolerance as a % of Sampling Interval --
            tolerance_percent = 0.25  # 25% of sampling interval

            # -- Compute Upper and Lower Bounds for Acceptable Trigger Delay --
            nearest_eq_df['Trigger Window Lower Bound [s]'] = nearest_eq_df['Surface Wave Travel Time [s]']
            nearest_eq_df['Trigger Window Upper Bound [s]'] = (
                nearest_eq_df['Surface Wave Travel Time [s]'] +
                nearest_eq_df['Sampling Interval [s]'] * (1 + tolerance_percent)
            )

            # -- Flag Events That Fall Within the Trigger Window --
            nearest_eq_df['Likely Triggered (Wave Arrival ± Tolerance)'] = (
                (nearest_eq_df['Time Since EQ [s]'] > nearest_eq_df['Trigger Window Lower Bound [s]']) &
                (nearest_eq_df['Time Since EQ [s]'] < nearest_eq_df['Trigger Window Upper Bound [s]'])
            )





            nearest_eq_df['Sampling Interval [s]'] = picks['Sampling rate, s'].values

            # Compute time delay between earthquake and creep event
            nearest_eq_df['Time Since EQ [s]'] = (nearest_eq_df['Start Time'] - nearest_eq_df['eq_time']).dt.total_seconds()

            # --- SURFACE WAVE TRIGGER LOGIC STARTS HERE ---

            

            nearest_eq_df[['Epicentral Distance [km]', 'Surface Wave Travel Time [s]']] = nearest_eq_df.apply(
                compute_epicentral_distance_and_travel_time, axis=1)

            # Define required delay after EQ: surface wave + 2×sampling interval
            nearest_eq_df['Min Required Delay [s]'] = (nearest_eq_df['Surface Wave Travel Time [s]'] + 2 * nearest_eq_df['Sampling Interval [s]'])

            # Determine if creep event occurs *after* that minimum delay
            nearest_eq_df['valid_trigger'] = nearest_eq_df['Time Since EQ [s]'] > nearest_eq_df['Min Required Delay [s]']

            # Compute percentage of events considered valid triggers
            count_valid = nearest_eq_df['valid_trigger'].sum()
            percent_valid = 100 * count_valid / len(picks)

            print(f"{percent_valid:.2f}% of {len(picks)} events have valid earthquake triggers (wave arrival + resolution buffer).")

            # Save summary stats
            percentage_triggered = pd.DataFrame({
                'Creepmeter': [abbrv],
                'percentage valid triggers': [round(percent_valid, 2)],
                'Latitude': lat
            })

            # Optional printout for debugging specific instruments
            #if abbrv in ['XHR1', 'XHR2', 'XHR3']:
                #print(nearest_eq_df[nearest_eq_df['valid_trigger'] == True])

            # Combine picks with matched EQ metadata
            creepmeter_eq_picks = pd.concat([picks.reset_index(drop=True), nearest_eq_df.reset_index(drop=True)], axis=1)
            my_data = pd.concat([my_data, creepmeter_eq_picks])
            percentage_triggered_all = pd.concat([percentage_triggered_all, percentage_triggered])

        # Sort summary table for this network by latitude (for map-style plotting)
        percentage_triggered_all.sort_values(by='Latitude', inplace=True, ascending=False)
        percentage_triggered_all.reset_index(inplace=True, drop=True)


EAF
BAL1
BAL1 37.9903 38.199
Picks with no backward match: 0
Picks with no forward match: 0
2
50.00% of 2 events have valid earthquake triggers (wave arrival + resolution buffer).
GOK1
GOK1 38.006 36.5267
Picks with no backward match: 0
Picks with no forward match: 0
130
46.92% of 130 events have valid earthquake triggers (wave arrival + resolution buffer).
GOZ1
GOZ1 38.1759 38.0103
Picks with no backward match: 0
Picks with no forward match: 0
1
0.00% of 1 events have valid earthquake triggers (wave arrival + resolution buffer).
HAS1
HAS1 36.8006 36.5185
Picks with no backward match: 0
Picks with no forward match: 0
8
75.00% of 8 events have valid earthquake triggers (wave arrival + resolution buffer).
HAT1
HAT1 38.387 36.2803
Picks with no backward match: 0
Picks with no forward match: 0
7
0.00% of 7 events have valid earthquake triggers (wave arrival + resolution buffer).
KAR1
KAR1 38.0697 38.4983
Picks with no backward match: 0
Picks with no forward match: 0
0


ValueError: Columns must be same length as key

In [11]:
my_data

,Network,Creepmeter full name,Creepmeter abbreviation,Latitude,Longitude,"Sampling rate, s",Start Time,End Time,"Displacement, mm","Duration, hrs",...,place,time_diff_seconds,Latitude_creep,Longitude_creep,sampling_interval_sec,time_since_eq_seconds,distance_km,travel_time_surf_wave,min_required_delay,valid_trigger
0,HOL,Cienega Winery Central,CWC3,36.750,-121.385,600.0,1992-01-07 06:00:00,1992-01-09 19:09:59,1.49,61.166389,...,"26km S of Trona, CA",1538490.620,36.750,-121.385,600.0,1538490.620,385.560590,110.160168,1310.160168,True
1,HOL,Cienega Winery Central,CWC3,36.750,-121.385,600.0,1992-01-21 20:39:59,1992-01-22 06:29:59,0.42,9.833333,...,"13 km SSW of Mountain House, California",2138180.650,36.750,-121.385,600.0,-2138180.650,104.952586,29.986453,1229.986453,False
2,HOL,Cienega Winery Central,CWC3,36.750,-121.385,600.0,1992-01-22 06:49:59,1992-01-23 23:39:59,1.38,40.833333,...,"13 km SSW of Mountain House, California",2101580.650,36.750,-121.385,600.0,-2101580.650,104.952586,29.986453,1229.986453,False
3,HOL,Cienega Winery Central,CWC3,36.750,-121.385,600.0,1992-01-24 08:10:00,1992-01-24 17:19:59,0.39,9.166389,...,"13 km SSW of Mountain House, California",1923979.650,36.750,-121.385,600.0,-1923979.650,104.952586,29.986453,1229.986453,False
4,HOL,Cienega Winery Central,CWC3,36.750,-121.385,600.0,1992-01-24 16:39:59,1992-01-26 00:29:59,0.74,31.833333,...,"13 km SSW of Mountain House, California",1893380.650,36.750,-121.385,600.0,-1893380.650,104.952586,29.986453,1229.986453,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOL,San Juan Bautista #3,XSJ3,36.836,-121.521,600.0,2015-07-02 01:39:59,2015-07-04 10:59:59,1.35,57.333333,...,"5 km WSW of Moskowite Corner, California",3538018.220,36.836,-121.521,600.0,3538018.220,188.437780,53.839366,1253.839366,True
66,HOL,San Juan Bautista #3,XSJ3,36.836,-121.521,600.0,2015-09-03 21:00:00,2015-09-04 03:29:59,0.26,6.499722,...,"4 km W of Walker Lake, Nevada",364519.980,36.836,-121.521,600.0,-364519.980,312.589879,89.311394,1289.311394,False
67,HOL,San Juan Bautista #3,XSJ3,36.836,-121.521,600.0,2015-09-04 06:40:00,2015-09-06 13:50:00,5.17,55.166667,...,"4 km W of Walker Lake, Nevada",329719.980,36.836,-121.521,600.0,-329719.980,312.589879,89.311394,1289.311394,False
68,HOL,San Juan Bautista #3,XSJ3,36.836,-121.521,600.0,2016-06-13 14:19:59,2016-06-14 03:29:59,0.12,13.166667,...,"3 km W of Walker Lake, Nevada",2105681.556,36.836,-121.521,600.0,-2105681.556,313.487369,89.567820,1289.567820,False


In [ ]:
my_data = pd.DataFrame()
percentage_triggered_all_rainfall = pd.DataFrame()
for j in range(len(networks)):
    print(networks[j])
    if networks[j] in ['HOL']:
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[j]].index)
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]            
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation']!=abbrv].index)
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv,lat,long)
            rainfall = pd.read_csv('../../Data/ECMWF/Creepmeter_precipitation/{p}/Precipitation_ECMWF_{q}_02_JUL_2024.csv'.format(p=networks[j],q=abbrv))

            # Ensure datetime format and sort
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            rainfall['rain_time'] = pd.to_datetime(rainfall['Time (UTC)'])

            picks = picks.sort_values('Start Time').reset_index(drop=True)
            rainfall = rainfall.sort_values('rain_time').reset_index(drop=True)

            # Set rain_time as index for fast slicing
            rainfall = rainfall.set_index('Time (UTC)')

            # Ensure the index is sorted
            rainfall = rainfall.sort_index()

            def rain_in_previous_hour(pick_time):
                pick_time = pd.to_datetime(pick_time)
                start = pick_time - pd.Timedelta(hours=1)
                # Clip start and end to be within the rainfall index range
                rainfall.index = pd.to_datetime(rainfall.index)
                start = max(start, rainfall.index.min())
                end = min(pick_time, rainfall.index.max())

                rain_window = rainfall.loc[start:end]
                if rain_window.empty:
                    return 0.0
                else:
                    return rain_window['Precipitation, m'].sum()

            picks['rain_last_hour'] = picks['Start Time'].apply(rain_in_previous_hour)
            picks['rain_before'] = picks['rain_last_hour'] > 0.0005
            
            if abbrv in ['XHR2']:
                boolarr = picks['rain_before']==True
                print(picks[boolarr])


            # Count how many are within 1 day
            count_within_hour = picks['rain_before'].sum()
    

            # Total number of entries
            total_count = len(picks)

            # Percentage
            percent_within_hour = 100 * count_within_hour / total_count

            #print(f"{percent_within_day:.2f}% of {total_count} events occur within 1 day of an earthquake.")
            print(f"{percent_within_hour:.2f}% of {total_count} events occur within 1 hour of 0.5mm of rainfall.")

            percentage_triggered = pd.DataFrame({'Creepmeter':[abbrv],'percentage in hour':[round(percent_within_hour,2)],
                                                    'Latitude':lat})


            percentage_triggered_all_rainfall = pd.concat([percentage_triggered_all_rainfall,percentage_triggered])
        percentage_triggered_all_rainfall.sort_values(by='Latitude',inplace=True,ascending=False)
        percentage_triggered_all_rainfall.reset_index(inplace=True,drop=True)








In [ ]:
cmap=cmc.managua

In [ ]:
colours  = cmap(np.linspace(0, 1, 10))

In [ ]:
plt.figure(figsize=(18,6))
ax = plt.subplot(1,1,1)
#plt.plot(percentage_triggered_all['Latitude'],percentage_triggered_all['percentage in day'])
# Add a rectangle: (x, y) = lower left corner; width and height
rect_Hawyard = patches.Rectangle((-0.5, 0), width=3, height=60, linewidth=2, edgecolor=colours[0], facecolor=colours[0],alpha=0.2)
rect_Calaveras = patches.Rectangle((2.5, 0), width=1, height=60, linewidth=2, edgecolor=colours[1], facecolor=colours[1],alpha=0.2)
rect_Hollister = patches.Rectangle((3.5, 0), width=10, height=60, linewidth=2, edgecolor=colours[2], facecolor=colours[2],alpha=0.2)
rect_Parkfield = patches.Rectangle((13.5, 0), width=15, height=60, linewidth=2, edgecolor=colours[4], facecolor=colours[4],alpha=0.2)
rect_SoCal = patches.Rectangle((28.5, 0), width=10, height=60, linewidth=2, edgecolor=colours[6], facecolor=colours[6],alpha=0.2)


plt.xticks(np.arange(len(percentage_triggered_all)),percentage_triggered_all['Creepmeter'],rotation=90)
# Add the patch to the Axes
ax.add_patch(rect_Hawyard)
ax.add_patch(rect_Calaveras)
ax.add_patch(rect_Hollister)
ax.add_patch(rect_Parkfield)
ax.add_patch(rect_SoCal)

plt.plot(np.arange(len(percentage_triggered_all)),percentage_triggered_all['percentage in hour'],
         color=colours[3],marker='o',label='Earthquake in hour prior')
plt.plot(np.arange(len(percentage_triggered_all_rainfall)),percentage_triggered_all_rainfall['percentage in hour'],
         color=colours[5],marker='^',label='Rainfall in hour prior')
plt.ylabel("% of events")
plt.xlabel('Creepmeter')
plt.legend()
plt.show()


In [ ]:
my_data = pd.DataFrame()
percentage_triggered_all_rainfall_boot = pd.DataFrame()
for j in range(len(networks)):
    print(networks[j])
    if networks[j] in ['SOCAL','HAY','HOL','CAL','PARK']:
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[j]].index)
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]            
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation']!=abbrv].index)
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv,lat,long)
            rainfall = pd.read_csv('../../Data/ECMWF/Creepmeter_precipitation/{p}/Precipitation_ECMWF_{q}_02_JUL_2024.csv'.format(p=networks[j],q=abbrv))

            # Ensure datetime format and sort
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            rainfall['rain_time'] = pd.to_datetime(rainfall['Time (UTC)'])

            picks = picks.sort_values('Start Time').reset_index(drop=True)
            rainfall = rainfall.sort_values('rain_time').reset_index(drop=True)

            # Set rain_time as index for fast slicing
            rainfall = rainfall.set_index('Time (UTC)')

            # Ensure the index is sorted
            rainfall = rainfall.sort_index()

            def rain_in_previous_hour(pick_time):
                pick_time = pd.to_datetime(pick_time)
                start = pick_time - pd.Timedelta(hours=1)
                #start = pick_time - pd.Timedelta(days = 1)
                # Clip start and end to be within the rainfall index range
                rainfall.index = pd.to_datetime(rainfall.index)
                start = max(start, rainfall.index.min())
                end = min(pick_time, rainfall.index.max())

                rain_window = rainfall.loc[start:end]
                if rain_window.empty:
                    return 0.0
                else:
                    return rain_window['Precipitation, m'].sum()

            picks['rain_last_hour'] = picks['Start Time'].apply(rain_in_previous_hour)
            picks['rain_before'] = picks['rain_last_hour'] > 0.0005
            
            # Total number of entries
            total_count = len(picks)
            
            bootstrapped = []
            for m in tqdm(range(1000)):
                selected = []
                for n in range(len(picks)):
                    rand_index = random.randint(0, len(picks) - 1)
                    selected.append(picks['rain_before'].iloc[rand_index])
                
                # Count how many are within 1 day
                count_within_hour = np.sum(selected)            

                # Percentage
                percent_within_hour = 100 * (count_within_hour / len(picks))
                bootstrapped.append(percent_within_hour)


            per_85 = np.percentile(bootstrapped,85)
            per_15 = np.percentile(bootstrapped,15)
            median = np.median(bootstrapped)

            percentage_triggered = pd.DataFrame({'Creepmeter':[abbrv],'per_85':[per_85],'per_15':[per_15],'median':[median],'Latitude':lat})


            percentage_triggered_all_rainfall_boot = pd.concat([percentage_triggered_all_rainfall_boot,percentage_triggered])
        percentage_triggered_all_rainfall_boot.sort_values(by='Latitude',inplace=True,ascending=False)
        percentage_triggered_all_rainfall_boot.reset_index(inplace=True,drop=True)

In [ ]:
percentage_triggered_all_rainfall_boot

In [ ]:
count_within_hour/total_count

In [ ]:
my_data = pd.DataFrame()
percentage_triggered_all_eq_boot = pd.DataFrame()
for j in tqdm(range(len(networks))):
    print(networks[j])
    if networks[j] in ['PARK','SOCAL','HAY','HOL','CAL']:
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[j]].index)
        for i in range(len(creepmeters_network)):#len(creepmeters)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]
            print(abbrv)
            
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation']!=abbrv].index)
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv,lat,long)


            # USGS API query parameters
            url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
            params = {
                "format": "geojson",
                "starttime": "1980-01-01",
                "endtime": "2024-01-01",
                "latitude": lat,
                "longitude": long,
                "maxradiuskm": 500,
                "minmagnitude": 4
            }

            response = requests.get(url, params=params)
            data = response.json()

            # Extract earthquake metadata
            eq_data = []
            for feature in data['features']:
                time = datetime.utcfromtimestamp(feature['properties']['time'] / 1000.0)
                magnitude = feature['properties']['mag']
                place = feature['properties']['place']
                lon_eq, lat_eq, depth = feature['geometry']['coordinates']  # [lon, lat, depth]
                eq_data.append({
                    'eq_time': time,
                    'magnitude': magnitude,
                    'latitude': lat_eq,
                    'longitude': lon_eq,
                    'depth_km': depth,
                    'place': place
                })

            # Create DataFrame
            eq_df = pd.DataFrame(eq_data)


            # Ensure both time columns are datetime
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            eq_df['eq_time'] = pd.to_datetime(eq_df['eq_time'])

            # Sort both by time (important!)
            picks = picks.sort_values('Start Time').reset_index(drop=True)
            eq_df = eq_df.sort_values('eq_time').reset_index(drop=True)

            # Step 1: Use merge_asof twice — once backward and once forward
            df_before = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='backward', suffixes=('', '_before'))
            # Forward merge
            df_after = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='forward')

            # Rename the columns to prevent confusion
            df_after = df_after.rename(columns={
                'eq_time': 'eq_time_after',
                'magnitude': 'magnitude_after',
                'latitude': 'latitude_after',
                'longitude': 'longitude_after',
                'depth_km': 'depth_km_after',
                'place': 'place_after'
            })


            # Step 2: Calculate time differences for each
            df_before['dt_before'] = (df_before['Start Time'] - df_before['eq_time']).dt.total_seconds().abs()
            df_after['dt_after'] = (df_after['Start Time'] - df_after['eq_time_after']).dt.total_seconds().abs()

            print("Picks with no backward match:", df_before['eq_time'].isna().sum())
            print("Picks with no forward match:", df_after['eq_time_after'].isna().sum())
            print(len(picks))

            # Step 3: Choose the closer one
            nearest_eq = []
            for i in range(len(picks)):
                dt_b = df_before.loc[i, 'dt_before']
                dt_a = df_after.loc[i, 'dt_after']
                
                # Use backward if forward is missing or farther
                if pd.isna(dt_a) or (not pd.isna(dt_b) and dt_b <= dt_a):
                    row = df_before.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time'],
                        'magnitude': row['magnitude'],
                        'latitude': row['latitude'],
                        'longitude': row['longitude'],
                        'depth_km': row['depth_km'],
                        'place': row['place'],
                        'time_diff_seconds': row['dt_before']
                    })
                else:
                    row = df_after.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time_after'],
                        'magnitude': row['magnitude_after'],
                        'latitude': row['latitude_after'],
                        'longitude': row['longitude_after'],
                        'depth_km': row['depth_km_after'],
                        'place': row['place_after'],
                        'time_diff_seconds': row['dt_after']
                    })

            # Step 4: Convert to DataFrame
            nearest_eq_df = pd.DataFrame(nearest_eq)

            # Add time difference column
            nearest_eq_df['time_since_eq_seconds'] = (nearest_eq_df['Start Time'] - nearest_eq_df['eq_time']).dt.total_seconds()

            # Threshold in seconds (1 day)
            one_day_seconds = 86400
            one_hour_seconds = 3600

            boolarr = abs(nearest_eq_df['time_since_eq_seconds']) < one_hour_seconds
            #boolarr = np.logical_and(nearest_eq_df['time_since_eq_seconds']>-3600,nearest_eq_df['time_since_eq_seconds'] < one_day_seconds)

            eq_trig = np.zeros(len(nearest_eq_df))
            eq_trig[boolarr] = 1

            nearest_eq_df['eq_trig'] = eq_trig
            bootstrapped = []
            for m in tqdm(range(1000)):
                selected = []
                for n in range(len(picks)):
                    rand_index = random.randint(0, len(picks) - 1)
                    selected.append(nearest_eq_df['eq_trig'].iloc[rand_index])
                
                # Count how many are within 1 day
                count_within_hour = np.sum(selected)     
                #print(count_within_hour,len(picks))       

                # Percentage
                percent_within_hour = 100 * (count_within_hour / len(picks))
                #print(percent_within_hour)
                bootstrapped.append(percent_within_hour)


            per_85 = np.percentile(bootstrapped,85)
            per_15 = np.percentile(bootstrapped,15)
            median = np.median(bootstrapped)

            percentage_triggered = pd.DataFrame({'Creepmeter':[abbrv],'per_85':[round(per_85,2)],'per_15':[per_15],'median':[median],'Latitude':lat})


            percentage_triggered_all_eq_boot = pd.concat([percentage_triggered_all_eq_boot,percentage_triggered])
        percentage_triggered_all_eq_boot.sort_values(by='Latitude',inplace=True,ascending=False)
        percentage_triggered_all_eq_boot.reset_index(inplace=True,drop=True)

            
            
            

In [ ]:
percentage_triggered_all_eq_boot

In [ ]:
plt.figure(figsize=(18,6))
ax = plt.subplot(1,1,1)
#plt.plot(percentage_triggered_all['Latitude'],percentage_triggered_all['percentage in day'])
# Add a rectangle: (x, y) = lower left corner; width and height
rect_Hawyard = patches.Rectangle((-0.5, -1), width=4, height=101, linewidth=2, edgecolor=colours[0], 
                                 facecolor=colours[0],alpha=0.2)
rect_Calaveras = patches.Rectangle((3.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[1], 
                                   facecolor=colours[1],alpha=0.2)
rect_Hollister = patches.Rectangle((4.5, -1), width=10, height=101, linewidth=2, edgecolor=colours[2], 
                                   facecolor=colours[2],alpha=0.2)
rect_Parkfield = patches.Rectangle((14.5, -1), width=15, height=101, linewidth=2, edgecolor=colours[4],
                                    facecolor=colours[4],alpha=0.2)
rect_SoCal = patches.Rectangle((29.5, -1), width=7, height=101, linewidth=2, edgecolor=colours[6], 
                               facecolor=colours[6],alpha=0.2)
rect_NS = patches.Rectangle((36.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[7], 
                            facecolor=colours[7],alpha=0.2)
rect_SU = patches.Rectangle((37.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[8], 
                            facecolor=colours[8],alpha=0.2)
rect_IM = patches.Rectangle((38.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[9], 
                            facecolor=colours[9],alpha=0.2)

plt.xticks(np.arange(len(percentage_triggered_all_eq_boot)),percentage_triggered_all_eq_boot['Creepmeter'],rotation=90)
# Add the patch to the Axes
ax.add_patch(rect_Hawyard)
ax.add_patch(rect_Calaveras)
ax.add_patch(rect_Hollister)
ax.add_patch(rect_Parkfield)
ax.add_patch(rect_SoCal)
ax.add_patch(rect_NS)
ax.add_patch(rect_SU)
ax.add_patch(rect_IM)
plt.hlines(0,-0.5,39.5,colors='k')

lower_err = percentage_triggered_all_eq_boot['median'] - percentage_triggered_all_eq_boot['per_15']
upper_err = percentage_triggered_all_eq_boot['per_85'] - percentage_triggered_all_eq_boot['median']

plt.errorbar(np.arange(len(percentage_triggered_all_eq_boot)), percentage_triggered_all_eq_boot['median'], 
             yerr=[lower_err, upper_err], 
             fmt='o', capsize=4,color=colours[3])

plt.plot(np.arange(len(percentage_triggered_all_eq_boot)),percentage_triggered_all_eq_boot['median'],
         color=colours[3],marker='o',label='Earthquake in hour prior')


lower_err = percentage_triggered_all_rainfall_boot['median'] - percentage_triggered_all_rainfall_boot['per_15']
upper_err = percentage_triggered_all_rainfall_boot['per_85'] - percentage_triggered_all_rainfall_boot['median']

plt.errorbar(np.arange(len(percentage_triggered_all_rainfall_boot)), percentage_triggered_all_rainfall_boot['median'], 
             yerr=[lower_err, upper_err], 
             fmt='o', capsize=4,color=colours[5])

plt.plot(np.arange(len(percentage_triggered_all_rainfall_boot)),percentage_triggered_all_rainfall_boot['median'],
         color=colours[5],marker='o',label='Rainfall in hour prior')

#plt.plot(np.arange(len(percentage_triggered_all_rainfall)),percentage_triggered_all_rainfall['percentage in hour'],
#         color=colours[5],marker='^',label='Rainfall in hour prior')
plt.ylabel("% of events")
plt.xlabel('Creepmeter')
plt.legend(loc='upper left', bbox_to_anchor=(0.01, 0.82))
plt.text(1.5,95,'Hayward Fault',verticalalignment='center',horizontalalignment='center')
plt.text(4,95,'Calaveras\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.text(9,95,'San Andreas Fault: Hollister',verticalalignment='center',horizontalalignment='center')
plt.text(21.5,95,'San Andreas Fault: Parkfield',verticalalignment='center',horizontalalignment='center')
plt.text(33,95,'San Andreas Fault: Salton Sea',verticalalignment='center',horizontalalignment='center')
plt.text(37,95,'North Shoreline\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.text(38,95,'Superstition Hills\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.text(39,95,'Imperial\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.xlim([-0.5,39.5])
plt.ylim([-1,100])
plt.tight_layout()
plt.savefig('../../Creep_catalog_stats_figures/Bootstrapped_percentages_events_rain_or_EQ_in_hour_prior.pdf')
plt.show()


In [ ]:
picks['Start Time']+dt.timedelta(days=365)

In [ ]:

from dateutil.relativedelta import relativedelta

def add_one_year(date_value):
  """Adds one year to a date, handling leap years correctly."""
  new_date = date_value + relativedelta(years=1)
  looping = int(str(picks['Start Time'].iloc[-1])[0:4]) - int(str(picks['Start Time'].iloc[0])[0:4])
  if new_date > dt.datetime(2024,1,1):
    new_date = new_date - relativedelta(years=looping+1)
  return new_date

In [ ]:
picks_test_circle = picks.copy(deep=True)

In [ ]:
for i in range(int(str(picks_test_circle['Start Time'].iloc[-1])[0:4]) - int(str(picks_test_circle['Start Time'].iloc[0])[0:4])):
    picks_test_circle['Start Time'] = picks_test_circle['Start Time'].apply(add_one_year)
    print(picks_test_circle)

In [ ]:
float(str(picks['Start Time'].iloc[0])[0:4])